In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
huggingface_token = os.getenv("HUGGINGFACE_TOKEN")
openai_token = os.getenv("OPENAI_API_KEY")

In [1]:
import re

def preprocess_text(text):
    # Fix common OCR patterns
    text = re.sub(r'[«»"“”‘’]', '', text)  # Remove quotation marks
    text = re.sub(r'\s+', ' ', text)       # Normalize spaces
    text = re.sub(r'[^a-zA-ZáéíóúñÁÉÍÓÚÑ0-9\s.,;:¿?¡!\-]', '', text)  # Remove special chars
    return text.strip()